# AutoML Classification experiment using Local Compute and Pandas DataFrames
## Data: IBM Employee Attrition dataset loaded from Azure ML Dataset

##  Get Azure ML Workspace to use

In [14]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

# Get Workspace defined in by default config.json file
ws = Workspace.from_config()

## Load data from Azure ML Datasets into Pandas DataFrame

In [15]:
# Load Data
aml_dataset = ws.datasets['IBM-Employee-Attrition']

# Use Pandas DataFrame just to sneak peak some data and schema
full_df = aml_dataset.to_pandas_dataframe()
# .to_pandas_dataframe().dropna()
full_df.head(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [16]:
# Use Pandas DataFrame just to investigate the dataset's schema and info
full_df.describe()

,Age,Attrition,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,...,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00,1470.00
mean,36.92,0.16,802.49,9.19,2.91,1.00,1024.87,2.72,65.89,2.73,...,2.71,80.00,0.79,11.28,2.80,2.76,7.01,4.23,2.19,4.12
std,9.14,0.37,403.51,8.11,1.02,0.00,602.02,1.09,20.33,0.71,...,1.08,0.00,0.85,7.78,1.29,0.71,6.13,3.62,3.22,3.57
min,18.00,0.00,102.00,1.00,1.00,1.00,1.00,1.00,30.00,1.00,...,1.00,80.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
25%,30.00,0.00,465.00,2.00,2.00,1.00,491.25,2.00,48.00,2.00,...,2.00,80.00,0.00,6.00,2.00,2.00,3.00,2.00,0.00,2.00
50%,36.00,0.00,802.00,7.00,3.00,1.00,1020.50,3.00,66.00,3.00,...,3.00,80.00,1.00,10.00,3.00,3.00,5.00,3.00,1.00,3.00
75%,43.00,0.00,1157.00,14.00,4.00,1.00,1555.75,4.00,83.75,3.00,...,4.00,80.00,1.00,15.00,3.00,3.00,9.00,7.00,3.00,7.00
max,60.00,1.00,1499.00,29.00,5.00,1.00,2068.00,4.00,100.00,4.00,...,4.00,80.00,3.00,40.00,6.00,4.00,40.00,18.00,15.00,17.00


## Clean up the initial dataset (Using related Pandas DataFrame)

In [17]:
# Dropping Employee count as all values are 1 and hence attrition is independent of this feature
full_df = full_df.drop(['EmployeeCount'], axis=1)
# Dropping Employee Number since it is merely an identifier
full_df = full_df.drop(['EmployeeNumber'], axis=1)

full_df = full_df.drop(['Over18'], axis=1)

# Since all values are 80
full_df = full_df.drop(['StandardHours'], axis=1)

## Split original dataset in test/train sets using Scikit-Learn train_test_split function

In [18]:
from sklearn.model_selection import train_test_split

# Split using ScikitLearn train_test_split function using Dataframes
# Will use test dataframe at the end, without AutoML, just for testing predictions with the model

# Only split in test/train
train_df, test_df = train_test_split(full_df, test_size=0.2, random_state=1)
train_df.describe()

# Split in x/y and test/train
# y_df = full_df.pop("Attrition")
# x_df = full_df
# x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.1, random_state=1)

#Another possibility would be to split using the Azure ML Datasets (Better for Remote Compute): 
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py#random-split-percentage--seed-none-

,Age,Attrition,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,...,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00,1176.00
mean,37.10,0.15,810.27,9.22,2.92,2.72,65.66,2.73,2.07,2.72,...,3.15,2.75,0.79,11.31,2.81,2.75,6.88,4.22,2.13,4.11
std,9.12,0.36,404.54,8.12,1.02,1.09,20.31,0.73,1.11,1.09,...,0.36,1.07,0.85,7.78,1.28,0.70,5.96,3.63,3.20,3.50
min,18.00,0.00,102.00,1.00,1.00,1.00,30.00,1.00,1.00,1.00,...,3.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
25%,30.00,0.00,473.50,2.00,2.00,2.00,48.00,2.00,1.00,2.00,...,3.00,2.00,0.00,6.00,2.00,2.00,3.00,2.00,0.00,2.00
50%,36.00,0.00,809.00,7.00,3.00,3.00,65.00,3.00,2.00,3.00,...,3.00,3.00,1.00,10.00,3.00,3.00,5.00,3.00,1.00,3.00
75%,43.00,0.00,1159.00,14.00,4.00,4.00,83.00,3.00,3.00,4.00,...,3.00,4.00,1.00,15.00,3.00,3.00,9.00,7.00,2.00,7.00
max,60.00,1.00,1499.00,29.00,5.00,4.00,100.00,4.00,5.00,4.00,...,4.00,4.00,3.00,40.00,6.00,4.00,40.00,18.00,15.00,17.00


In [19]:
test_df.describe()

,Age,Attrition,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,...,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00
mean,36.23,0.20,771.34,9.08,2.87,2.71,66.81,2.75,2.04,2.75,...,3.16,2.57,0.82,11.17,2.74,2.81,7.51,4.26,2.40,4.18
std,9.17,0.40,398.54,8.07,1.05,1.10,20.42,0.65,1.10,1.14,...,0.37,1.13,0.87,7.81,1.31,0.72,6.75,3.62,3.30,3.85
min,18.00,0.00,115.00,1.00,1.00,1.00,30.00,1.00,1.00,1.00,...,3.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
25%,30.00,0.00,427.50,2.00,2.00,2.00,50.00,2.00,1.00,2.00,...,3.00,2.00,0.00,6.00,2.00,2.00,3.00,2.00,0.00,1.25
50%,35.00,0.00,726.50,7.00,3.00,3.00,68.50,3.00,2.00,3.00,...,3.00,3.00,1.00,10.00,3.00,3.00,5.00,3.00,1.00,3.00
75%,42.00,0.00,1146.00,14.00,4.00,4.00,84.75,3.00,2.00,4.00,...,3.00,4.00,1.00,15.00,3.00,3.00,10.00,7.00,4.00,7.00
max,59.00,1.00,1498.00,29.00,5.00,4.00,100.00,4.00,5.00,4.00,...,4.00,4.00,3.00,37.00,6.00,4.00,36.00,18.00,15.00,17.00


## List and select primary metric to drive the AutoML classification problem

In [20]:
from azureml.train import automl

# List of possible primary metrics is here:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#primary-metric
    
# Get a list of valid metrics for your given task
automl.utilities.get_primary_metrics('classification')

# I'll use 'accuracy' as primary metric (Closer to 1.00 is better)

['accuracy',
 'average_precision_score_weighted',
 'norm_macro_recall',
 'precision_score_weighted',
 'AUC_weighted']

## Define AutoML Experiment settings

In [21]:
import logging

# Explanation of Settings: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#configure-your-experiment-settings

# AutoMLConfig info on: 
# https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig

# You can provide additional settings as a **kwargs parameter for the AutoMLConfig object
# automl_settings = {
#     "whitelist_models": 'XGBoostClassifier'
# }

from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='classification',
                             primary_metric='accuracy',
                             # experiment_timeout_minutes= 20,                            
                             training_data=train_df,
                             label_column_name="Attrition",
                             # X=x_train.values,             # X parameter is deprecated 
                             # y=y_train.values.flatten(),   # y parameter is deprecated 
                             n_cross_validations= 5,
                             # blacklist_models='XGBoostClassifier', 
                             # iteration_timeout_minutes= 5,                                                    
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log='automated_ml_errors.log',
                             verbosity= logging.INFO,
                             # **automl_settings
                             )

# WARNING: If using X and y parameters (deprecated) you get the following warning
# WARNING - The AutoMLConfig inputs you have specified will soon be deprecated. Please use the AutoMLConfig shown in our documentation: https://aka.ms/AutoMLConfig


## Run Experiment with multiple child runs under the covers

In [22]:
from azureml.core import Experiment
from datetime import datetime

now = datetime.now()
time_string = now.strftime("%m-%d-%Y-%H")
#time_string = now.strftime("%m-%d-%Y-%H-%M")
print(time_string)
experiment_name = "classif-automl-local-{0}".format(time_string)
print(experiment_name)

experiment = Experiment(workspace=ws, 
                        name=experiment_name)

run = experiment.submit(automl_config, show_output=True)

01-12-2020-21
classif-automl-local-01-12-2020-21
Running on local machine
Parent Run ID: AutoML_0ec54b76-21b0-4bb2-9de6-be311285ba89

Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS SUMMARY:
For more details, use API: run.get_guardrails()

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Classes are balanced in the training data.

TYPE:         Missing values imputation
STATUS:       PASSED
DESCRIPTION:  There were no missing values found in the training data.

TYPE:         High cardinality feature detection
STAT

## Explore results with Widget

In [23]:
# Explore the results of automatic training with a Jupyter widget: https://docs.microsoft.com/en-us/python/api/azureml-widgets/azureml.widgets?view=azure-ml-py
from azureml.widgets import RunDetails
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET', …

## Retrieve the 'Best Model'

In [25]:
best_run, fitted_model = run.get_output()
print(best_run)
print('--------')
print(fitted_model)

Run(Experiment: classif-automl-local-01-12-2020-21,
Id: AutoML_0ec54b76-21b0-4bb2-9de6-be311285ba89_30,
Type: None,
Status: Completed)
--------
Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, is_cross_validation=None,
        is_onnx_compatible=None, logger=None, observer=None, task=None)), ('pref...  flatten_transform=None,
               weights=[0.125, 0.125, 0.125, 0.125, 0.125, 0.25, 0.125]))])


## See files associated with the 'Best run'

In [26]:
print(best_run.get_file_names())

# best_run.download_file('azureml-logs/70_driver_log.txt')

['accuracy_table', 'confusion_matrix', 'outputs/conda_env_v_1_0_0.yml', 'outputs/env_dependencies.json', 'outputs/model.pkl', 'outputs/scoring_file_v_1_0_0.py', 'pipeline_graph.json']


## Download run model .pkl file and related files to see Conda and Environment dependencies used by AutoML

In [27]:
# Download the run's files
best_run.download_file('outputs/model.pkl')
best_run.download_file('outputs/conda_env_v_1_0_0.yml')
best_run.download_file('outputs/env_dependencies.json')
best_run.download_file('outputs/scoring_file_v_1_0_0.py')
best_run.download_file('pipeline_graph.json')

## Register the model
Once you've trained the model, you can save and register it to your workspace. Model registration lets you store and version your models in your workspace to simplify model management and deployment.

Running the following code will register the model to your workspace, and will make it available to reference by name in remote compute contexts or deployment scripts. 

In [34]:
from azureml.core.model import Model

model_reg = best_run.register_model(model_name='classif-empl-automl-local',      # Name of the registered model in your workspace.
                                    description='Binary classification model for employees attrition. From AutoML local training',
                                    model_path='outputs/model.pkl',              # Local file to upload and register as a model.
                                    model_framework=Model.Framework.SCIKITLEARN, # Framework used to create the model.
                                    model_framework_version='0.XX.X',            # Version of scikit-learn used to create the model.
                                    tags={'ml-task': "binary-classification", 'business-area': "HR"},
                                    properties={'pandas-version': "0.XX.X"},
                                    sample_input_dataset=aml_dataset
                              )

print(model_reg)

# (Q1:) How can we know what frameworks/libraries versions were used by AutoML when training the model?
#         - If investigating the outputs/conda_env_v_1_0_0.yml file, some libraries like Pandas and Scikit-Learn are not showing any version...

# (Q2:) Why best_run.get_environment() fails?
# best_run_environment = best_run.get_environment() 
# print(best_run_environment)


Model(workspace=Workspace.create(name='cesardl-automl-northcentralus-ws', subscription_id='102a16c3-37d3-48a8-9237-4c9b1e8e80e0', resource_group='automlpmdemo'), name=classif-empl-automl-local, id=classif-empl-automl-local:7, version=7, tags={'ml-task': 'binary-classification', 'business-area': 'HR'}, properties={'pandas-version': '0.XX.X'})


## Make Predictions

### Extract X values (feature columns) from test dataset and convert to NumPi array for predicting 

In [35]:
import pandas as pd

#Remove Label/y column
if 'Attrition' in test_df.columns:
    y_test_df = test_df.pop('Attrition')

x_test_df = test_df

In [36]:
x_test_df.describe()

,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,...,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00,294.00
mean,36.23,771.34,9.08,2.87,2.71,66.81,2.75,2.04,2.75,6448.35,...,3.16,2.57,0.82,11.17,2.74,2.81,7.51,4.26,2.40,4.18
std,9.17,398.54,8.07,1.05,1.10,20.42,0.65,1.10,1.14,4769.72,...,0.37,1.13,0.87,7.81,1.31,0.72,6.75,3.62,3.30,3.85
min,18.00,115.00,1.00,1.00,1.00,30.00,1.00,1.00,1.00,1052.00,...,3.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00
25%,30.00,427.50,2.00,2.00,2.00,50.00,2.00,1.00,2.00,2934.50,...,3.00,2.00,0.00,6.00,2.00,2.00,3.00,2.00,0.00,1.25
50%,35.00,726.50,7.00,3.00,3.00,68.50,3.00,2.00,3.00,4838.00,...,3.00,3.00,1.00,10.00,3.00,3.00,5.00,3.00,1.00,3.00
75%,42.00,1146.00,14.00,4.00,4.00,84.75,3.00,2.00,4.00,7551.50,...,3.00,4.00,1.00,15.00,3.00,3.00,10.00,7.00,4.00,7.00
max,59.00,1498.00,29.00,5.00,4.00,100.00,4.00,5.00,4.00,19999.00,...,4.00,4.00,3.00,37.00,6.00,4.00,36.00,18.00,15.00,17.00


### Make the actual Predictions

#### (Optional) Download Model from Registry and load in-memory

In [37]:
print(Model.get_model_path('classif-empl-automl-local', _workspace=ws))

model_definition_from_registry = Model(ws,'classif-empl-automl-local')
model_definition_from_registry.download(target_dir='.', exist_ok=True)
print(model_definition_from_registry)
print('-------')

# Load the model into memory
import joblib
fitted_model = joblib.load('model.pkl')
print(fitted_model)

azureml-models/classif-empl-automl-local/7/model.pkl
Model(workspace=Workspace.create(name='cesardl-automl-northcentralus-ws', subscription_id='102a16c3-37d3-48a8-9237-4c9b1e8e80e0', resource_group='automlpmdemo'), name=classif-empl-automl-local, id=classif-empl-automl-local:7, version=7, tags={'ml-task': 'binary-classification', 'business-area': 'HR'}, properties={'pandas-version': '0.XX.X'})
-------
Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, is_cross_validation=None,
        is_onnx_compatible=None, logger=None, observer=None, task=None)), ('pref...  flatten_transform=None,
               weights=[0.125, 0.125, 0.125, 0.125, 0.125, 0.25, 0.125]))])


In [40]:
# Try the best model making predictions with the test dataset
y_predictions = fitted_model.predict(x_test_df)

print('10 predictions: ')
print(y_predictions[:10])

10 predictions: 
[0 1 1 1 0 0 0 0 0 0]


In [41]:
y_predictions.shape

(294,)

### Calculate the Accuracy with Test Dataset (Not used for training)

In [42]:
from sklearn.metrics import accuracy_score

print('Accuracy:')
accuracy_score(y_test_df, y_predictions)

Accuracy:


0.8605442176870748